## Imports

In [30]:
from tensorflow.keras.utils import image_dataset_from_directory
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import inception_resnet_v2
import matplotlib.pyplot as plt
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D, MaxPool2D, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import pickle

## Load Data

In [31]:
# Data from google colab
#from google.colab import drive
#drive.mount('/content/gdrive')

In [32]:
#Import from local repository

data = image_dataset_from_directory("/Users/cristobalmorano/code/nadiasalmen/waiste/waiste/data", 
                             labels='inferred', label_mode='categorical', class_names=None, color_mode='rgb',
                             batch_size=32, image_size=(128, 128), shuffle=True, seed=None,
                             validation_split=None, subset=None, interpolation='bilinear', follow_links=False,
                             crop_to_aspect_ratio=False)

Found 22610 files belonging to 10 classes.


In [33]:
#Take number of images according to batch size

data_sample = data.take(70)

data_im = []
data_lb = []

for im, lb in data_sample:
    data_im.append(im)
    data_lb.append(lb)

2021-09-28 20:23:20.369471: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


In [34]:
#Normalize image tensors (0 - 1)

def norma_2(image):
    image = tf.cast(image/255, tf.float32)
    return image

data_im_2 = [norma_2(image) for image in data_im]

In [35]:
#Convert X and y to numpy arrays

data_im_2 = np.array(data_im_2)
data_lb = np.array(data_lb)

In [36]:
#Reshape for model

data_im_2 = data_im_2.reshape(-1, 128, 128, 3)
data_lb = data_lb.reshape(-1, 10)

In [37]:
# Split the data

train_X, test_X, train_y, test_y = train_test_split(data_im_2, data_lb, test_size=0.2, random_state=42)

## Build model

In [38]:
#Import InceptionResNetV2

inception_resnet_v2 = inception_resnet_v2.InceptionResNetV2(include_top=False, 
                                                            weights="imagenet", input_tensor=None, 
                                                            input_shape=(128, 128, 3), pooling=None, 
                                                            classifier_activation='softmax')

In [39]:
inception_resnet_v2.trainable = False

In [43]:
def init_model():
    model = Sequential()
    model.add(Input(shape=(128, 128, 3)))
    
    model.add(inception_resnet_v2)
    
    model.add(Conv2D(32, (4,4), padding='same', activation="relu"))
    model.add(MaxPool2D(pool_size=(4, 4), padding='same'))
    model.add(Dropout(0.3))   
    
    model.add(Conv2D(16, (3,3), padding='same', activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2), padding='same'))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
    return model

In [44]:
model = init_model()

In [45]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Functio (None, 2, 2, 1536)        54336736  
_________________________________________________________________
conv2d_426 (Conv2D)          (None, 2, 2, 32)          786464    
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 1, 1, 32)          0         
_________________________________________________________________
dropout_24 (Dropout)         (None, 1, 1, 32)          0         
_________________________________________________________________
conv2d_427 (Conv2D)          (None, 1, 1, 16)          4624      
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 1, 1, 16)          0         
_________________________________________________________________
dropout_25 (Dropout)         (None, 1, 1, 16)         

In [46]:
es = EarlyStopping(patience = 25, monitor='val_loss' , min_delta=0.001, restore_best_weights = True)

In [47]:
history = model.fit(train_X, train_y, validation_split = 0.3, 
                    callbacks = [es], batch_size = 64, 
                    epochs = 500, verbose = 1)

Epoch 1/500
20/20 [==============================] - 67s 3s/step - loss: 2.2881 - accuracy: 0.1611 - val_loss: 2.1055 - val_accuracy: 0.3086
Epoch 2/500
20/20 [==============================] - 53s 3s/step - loss: 2.0999 - accuracy: 0.2185 - val_loss: 1.9789 - val_accuracy: 0.3699
Epoch 3/500
20/20 [==============================] - 53s 3s/step - loss: 1.9417 - accuracy: 0.3054 - val_loss: 1.8206 - val_accuracy: 0.4665
Epoch 4/500
20/20 [==============================] - 87s 4s/step - loss: 1.8065 - accuracy: 0.3724 - val_loss: 1.6978 - val_accuracy: 0.4647
Epoch 5/500
20/20 [==============================] - 63s 3s/step - loss: 1.6768 - accuracy: 0.4219 - val_loss: 1.5341 - val_accuracy: 0.5093
Epoch 6/500
20/20 [==============================] - 60s 3s/step - loss: 1.5037 - accuracy: 0.4936 - val_loss: 1.3883 - val_accuracy: 0.5874
Epoch 7/500
20/20 [==============================] - 57s 3s/step - loss: 1.3979 - accuracy: 0.5263 - val_loss: 1.3681 - val_accuracy: 0.5967
Epoch 8/500
2

In [ ]:
def summarize_diagnostics(history):
    # plot loss
    plt.figure(figsize=(10, 15))
    plt.subplot(211)
    plt.title('Cross Entropy Loss')
    plt.plot(history.history['loss'], color='blue', label='train')
    plt.plot(history.history['val_loss'], color='orange', label='test')
    plt.legend()
    # plot accuracy
    plt.subplot(212)
    plt.title('Classification Accuracy')
    plt.plot(history.history['accuracy'], color='blue', label='train')
    plt.plot(history.history['val_accuracy'], color='orange', label='test')
    plt.legend()

In [ ]:
summarize_diagnostics(history)

In [ ]:
scores = model.evaluate(test_X, test_y, verbose=1)

In [ ]:
tf.keras.models.save_model(
    model, filepath, overwrite=True, include_optimizer=True, save_format=None,
    signatures=None, options=None, save_traces=True
)